<a href="introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# lambda式とmap, filter, reduce

ここでは以下についてやります。

- ラムダ式ってなんぞ？
- map, filter, reduceの使い方

## lambda式

WIP

## map, filter, reduce

WIP

- [Pythonのreduceと内包表記／ジェネレータ式を比較してみた | DevelopersIO](https://dev.classmethod.jp/articles/python-reduce-vs-generator)